# Nimbus ideal vs. payoff ideal

There is some issues that cause the results attained by Nimbus method to be higher than the ideal values attained by calculating the payoff table for the clusters.

In [1]:
from interactiveBoreal import *
import gradutil
from BorealWeights import BorealWeightedProblem

In [2]:
kehys = ReferenceFrame()
nclust = 600
kehys.cluster(nclust=nclust, seedn=6, outdata=kehys.x_stack)
data = kehys.centers
nvar = len(kehys.x_norm)
weights_norm = kehys.weights/nvar
ideal = kehys.ideal
nadir = kehys.nadir

In [3]:
solver_name = 'cplex'

Now we try to calculate ideal and possible nadir values for all the objectives using nimbus scalarization:

In [19]:
nimbus_ref = np.array([[kehys.ideal[0], kehys.nadir[1], kehys.nadir[2], kehys.nadir[3]],
                       [kehys.nadir[0], kehys.ideal[1], kehys.nadir[2], kehys.nadir[3]],
                       [kehys.nadir[0], kehys.nadir[1], kehys.ideal[2], kehys.nadir[3]],
                       [kehys.nadir[0], kehys.nadir[1], kehys.nadir[2], kehys.ideal[3]]])
stay = np.array([], dtype=int)
detoriate = np.array([], dtype=int)
nimbus_res_cluster = []
nimbus_res_orig = []
nimbus_problems = []
ide = kehys.normalize_ref(kehys.ideal)
nad = kehys.normalize_ref(kehys.nadir)
for i in range(len(nimbus_ref)):
    minmax = np.array([i], dtype=int)
    ref = kehys.normalize_ref(nimbus_ref[i])
    nimbus_problems.append(NIMBUS(ide, nad, ref, kehys.centers, 
                                  minmax, stay, detoriate, np.array([0,0,0,0]), 
                                  weights=kehys.weights, nvar=nvar))
    nimbus_solver = Solver(nimbus_problems[i].model, solver=solver_name)
    res = nimbus_solver.solve() 
    nimbus_res_cluster.append([gradutil.model_to_real_values(kehys.x_stack[:,:,j], 
                                                             nimbus_problems[i].model, 
                                                             kehys.xtoc) 
                               for j in range(len(nimbus_ref))])
    nimbus_res_orig.append([gradutil.cluster_to_value(kehys.x_stack[:,:,j], 
                                                      gradutil.res_to_list(nimbus_problems[i].model), 
                                                      kehys.weights)
                            for j in range(len(nimbus_ref))])

In [21]:
for k in nimbus_res_cluster:
     print(['{:12.2f}'.format(value) for value in k])

As we can see, the greatest values column wise are at the diagonal. As NIMBUS uses exactly these in its operations this seems very good!

In [22]:
for k in nimbus_res_orig:
    print(['{:12.2f}'.format(value) for value in k])

Now when we print the original variable values, we see that the biggest values are not anymore at the diagonal. This was expected so no need to worry

Ideal of the attained results:


In [23]:
nimbus_ideal = np.max(nimbus_res_cluster, axis=0)
print(['{:12.2f}'.format(value) for value in nimbus_ideal])

And the nadir

In [24]:
nimbus_nadir = np.min(nimbus_res_cluster, axis=0)
print(['{:12.2f}'.format(value) for value in nimbus_nadir])

In [25]:
print(['{:12.2f}'.format(value) for value in kehys.ideal])
print(['{:12.2f}'.format(value) for value in kehys.nadir])

The differences are then not so big and so NIMBUS seems to work!